In [1]:
import gspread
import pandas as pd
import os
import numpy as np

from dotenv import load_dotenv
from google.oauth2 import service_account

# === Carregar variáveis de ambiente ===
load_dotenv()

# === Definir escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

# === Caminho seguro para o arquivo JSON (lido do .env) ===
cred_path = os.getenv('GOOGLE_CREDENTIALS_PATH')

# === Carregar credenciais de forma moderna ===
creds = service_account.Credentials.from_service_account_file(
    cred_path,
    scopes=scopes
)

# === Autorizar o cliente gspread ===
client = gspread.authorize(creds)

# === Função para carregar aba da planilha ===
def carregar_aba(sheet_id, aba_nome):
    planilha = client.open_by_key(sheet_id)
    aba = planilha.worksheet(aba_nome)
    dados = aba.get_all_records()
    return pd.DataFrame(dados)

# === IDs das planilhas (fixar no .env depois se quiser) ===
id_dados_leads_ssp = "1K7nLxfgu6ktdyFwlHMPJsj5xdv410tensfPccmmPDm8"
id_pesquisa_ssp = "1pK0BP8n98DtQ8OwGeBhzcGgxQinFqZhZsQTk7_cYZIk"
id_compradores_ssp = "1A3D8JNPlZNpxkwcXQ_7CxalNEZyLo8KmJP6Sa_A-3NU"

# === Carregar DataFrames ===
df_leads_trafego = carregar_aba(id_dados_leads_ssp, "Página1")
df_leads_invest_trafego = carregar_aba(id_dados_leads_ssp, "Dados Investimento")
df_pesquisa_captacao = carregar_aba(id_pesquisa_ssp, "Página1")
df_compradores = carregar_aba(id_compradores_ssp, "Compradores")

In [2]:
df_pesquisa_captacao = df_pesquisa_captacao.drop(["Funil"], axis=1, errors='ignore')
df_pesquisa_captacao.rename(columns={'E-mail': 'email'}, inplace=True)
df_pesquisa_captacao.columns = df_pesquisa_captacao.columns.str.lower()

df_pesquisa_captacao['data'] = pd.to_datetime(df_pesquisa_captacao['data'], errors='coerce')
df_pesquisa_captacao['data'] = df_pesquisa_captacao['data'].dt.tz_localize(None)
df_pesquisa_captacao['data'] = df_pesquisa_captacao['data'].dt.normalize()

df_pesquisa_captacao["email"] = df_pesquisa_captacao["email"].str.lower().str.strip()
df_pesquisa_captacao = df_pesquisa_captacao.drop_duplicates(subset='email', keep='first')

df_pesquisa_captacao.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29024 entries, 0 to 29838
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   29024 non-null  datetime64[ns]
 1   email                  29024 non-null  object        
 2   idade                  29024 non-null  object        
 3   nível                  29024 non-null  object        
 4   situação profissional  29024 non-null  object        
 5   renda                  29024 non-null  object        
 6   escolaridade           29024 non-null  object        
 7   duvida                 29024 non-null  object        
 8   dificuldade            29024 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 2.2+ MB


In [3]:
df_pesquisa_captacao["renda"] = (
    df_pesquisa_captacao["renda"]
    .replace({
        "O a R$1.000\u2028": "De 0 a 1.000",
        "R$1.000 a R$3.000": "De 1.000 a 3.000",
        "R$3.000 a R$5.000\u2028": "De 3.000 a 5.000",
        "R$5.000 a R$10.000\u2028": "De 5.000 a 10.000",
        "Mais de R$10.000": "Mais de 10.000"
    })
)

df_pesquisa_captacao["escolaridade"] = (
    df_pesquisa_captacao["escolaridade"]
    .replace({
        "Ensino superior incompleto\u2028": "Ensino superior incompleto"
    })
)

In [4]:
# Criar coluna de lançamentos baseada na data
df_pesquisa_captacao['lancamentos'] = df_pesquisa_captacao['data'].apply(
    lambda x: 'SSP-L13' if x.date() >= pd.to_datetime('2025-04-30').date() else 'SSP-L12'
)

# Remover e-mails duplicados dentro de cada lançamento
df_pesquisa_captacao = (
    df_pesquisa_captacao
    .sort_values('data')  # garante ordem cronológica
    .drop_duplicates(subset=['email', 'lancamentos'], keep='first')
)

df_pesquisa_captacao.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29024 entries, 0 to 29838
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   29024 non-null  datetime64[ns]
 1   email                  29024 non-null  object        
 2   idade                  29024 non-null  object        
 3   nível                  29024 non-null  object        
 4   situação profissional  29024 non-null  object        
 5   renda                  29024 non-null  object        
 6   escolaridade           29024 non-null  object        
 7   duvida                 29024 non-null  object        
 8   dificuldade            29024 non-null  object        
 9   lancamentos            29024 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 2.4+ MB


In [5]:
# Filtrar o DataFrame para excluir linhas onde o email contém "teste" ou "x@x"
df_pesquisa_captacao = df_pesquisa_captacao[
    ~df_pesquisa_captacao['email'].str.contains('teste|x@x', case=False, na=False)
]

df_pesquisa_captacao.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,lancamentos
1073,2025-04-07,38-45,Técnica,Já sou funcionário terceirizado do sistema Pet...,De 3.000 a 5.000,Nível técnico completo,Como participar do processo seletivo,Tempo,SSP-L12
1072,2025-04-07,38-45,Técnica,Desempregado(a),Estou desempregado(a),Nível técnico completo,Assunto específico que cai na prova?,Dificuldade em aprender!,SSP-L12
1071,2025-04-07,28-32,Técnica,Funcionário público,De 1.000 a 3.000,Ensino médio completo,Nas perguntas,Concentração,SSP-L12
1070,2025-04-07,18-22,Técnica,Desempregado(a),Estou desempregado(a),Ensino médio completo,Como estudar,Constância,SSP-L12
1069,2025-04-07,23-27,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino médio completo,Qual tempo de curso,Nenhuma,SSP-L12


In [6]:
df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47109 entries, 0 to 47108
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Data          47109 non-null  object
 1   Nome          47109 non-null  object
 2   E-mail        47109 non-null  object
 3   UTM Source    47109 non-null  object
 4   UTM Campaign  47109 non-null  object
 5   UTM Medium    47109 non-null  object
 6   UTM Content   47109 non-null  object
 7   Tag           47109 non-null  object
dtypes: object(8)
memory usage: 2.9+ MB


In [7]:
df_leads_trafego.columns = df_leads_trafego.columns.str.strip().str.lower().str.replace(" ", "_")

# Definir colunas UTM
colunas_utms = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']

# Função para verificar se uma célula é "vazia de verdade"
def is_vazio(valor):
    if pd.isna(valor):
        return True
    valor = str(valor).strip()
    return (valor == "") or (valor.startswith("{{") and valor.endswith("}}"))

# Função para verificar se TODAS UTMs estão vazias para uma linha
def todas_utms_vazias(row):
    return all(is_vazio(row[col]) for col in colunas_utms)

# Aplicar filtro: manter linhas onde pelo menos uma UTM está preenchida de verdade
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego.apply(todas_utms_vazias, axis=1)
].copy()

# Remover linhas que contenham 'teste' em qualquer coluna
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego.apply(lambda row: row.astype(str).str.lower().str.contains('teste').any(), axis=1)
]

# Resetar índice
df_leads_trafego = df_leads_trafego.reset_index(drop=True)

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32494 entries, 0 to 32493
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   data          32494 non-null  object
 1   nome          32494 non-null  object
 2   e-mail        32494 non-null  object
 3   utm_source    32494 non-null  object
 4   utm_campaign  32494 non-null  object
 5   utm_medium    32494 non-null  object
 6   utm_content   32494 non-null  object
 7   tag           32494 non-null  object
dtypes: object(8)
memory usage: 2.0+ MB


In [8]:
df_leads_trafego.rename(columns={'e-mail': 'email'}, inplace=True)
df_leads_trafego["email"] = df_leads_trafego["email"].str.lower().str.strip()

df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')
df_leads_trafego['data'] = df_leads_trafego['data'].dt.tz_localize(None)
df_leads_trafego['data'] = df_leads_trafego['data'].dt.normalize()

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32494 entries, 0 to 32493
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          32494 non-null  datetime64[ns]
 1   nome          32494 non-null  object        
 2   email         32494 non-null  object        
 3   utm_source    32494 non-null  object        
 4   utm_campaign  32494 non-null  object        
 5   utm_medium    32494 non-null  object        
 6   utm_content   32494 non-null  object        
 7   tag           32494 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 2.0+ MB


C:\Users\Camilo_Bica\AppData\Local\Temp\ipykernel_15540\1702501422.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')


In [9]:
# Criar coluna de lançamentos baseada na data
df_leads_trafego['lancamentos'] = df_leads_trafego['data'].apply(
    lambda x: 'SSP-L13' if x.date() >= pd.to_datetime('2025-04-30').date() else 'SSP-L12'
)

# Remover e-mails duplicados dentro de cada lançamento
df_leads_trafego = (
    df_leads_trafego
    .sort_values('data')  # garante ordem cronológica
    .drop_duplicates(subset=['email', 'lancamentos'], keep='first')
)

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32438 entries, 0 to 32493
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          32438 non-null  datetime64[ns]
 1   nome          32438 non-null  object        
 2   email         32438 non-null  object        
 3   utm_source    32438 non-null  object        
 4   utm_campaign  32438 non-null  object        
 5   utm_medium    32438 non-null  object        
 6   utm_content   32438 non-null  object        
 7   tag           32438 non-null  object        
 8   lancamentos   32438 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 2.5+ MB


In [10]:
df_leads_trafego.drop(columns=['email']).head()

,data,nome,utm_source,utm_campaign,utm_medium,utm_content,tag,lancamentos
0,2025-03-31,Hermes,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_002_ANUNCIO PETRO 02,SSP-L12,SSP-L12
300,2025-03-31,Johnny,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12
299,2025-03-31,Willian,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12
297,2025-03-31,Robson,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12
296,2025-03-31,Zidane,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12


In [11]:
df_leads_invest_trafego['date'] = pd.to_datetime(df_leads_invest_trafego['date'], errors='coerce')
df_leads_invest_trafego['date'] = df_leads_invest_trafego['date'].dt.tz_localize(None)
df_leads_invest_trafego['date'] = df_leads_invest_trafego['date'].dt.normalize()

df_leads_invest_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       788 non-null    datetime64[ns]
 1   campaign                   788 non-null    object        
 2   adset_name                 788 non-null    object        
 3   ad_name                    788 non-null    object        
 4   spend                      788 non-null    int64         
 5   impressions                788 non-null    int64         
 6   actions_link_click         788 non-null    int64         
 7   actions_landing_page_view  788 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 49.4+ KB


C:\Users\Camilo_Bica\AppData\Local\Temp\ipykernel_15540\352342578.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_leads_invest_trafego['date'] = pd.to_datetime(df_leads_invest_trafego['date'], errors='coerce')


In [12]:
# Lista das colunas que queremos validar
colunas_investimento = ['spend', 'impressions', 'actions_link_click', 'actions_landing_page_view']

# Garantir que essas colunas sejam numéricas (importante!)
for col in colunas_investimento:
    df_leads_invest_trafego[col] = (
        df_leads_invest_trafego[col]
        .astype(str)
        .str.replace(',', '.', regex=False)
        .astype(float)
    )

# Agora, remover linhas onde TODAS essas colunas são zero
df_leads_invest_trafego = df_leads_invest_trafego[
    ~(df_leads_invest_trafego[colunas_investimento] == 0).all(axis=1)
].copy()

# Ajustar df_leads_invest_trafego para nomes iguais
df_leads_invest_trafego = df_leads_invest_trafego.rename(columns={
    'date': 'data',
    'campaign': 'utm campaign',
    'adset_name': 'utm medium',
    'ad_name': 'utm content'
})

# Resetar índice (opcional, para organização)
df_leads_invest_trafego = df_leads_invest_trafego.reset_index(drop=True)

In [13]:
df_leads_invest_trafego

,data,utm campaign,utm medium,utm content,spend,impressions,actions_link_click,actions_landing_page_view
0,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_003_ANUNCIO PETRO 03,4004.0,2878.0,16.0,1.0
1,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_002_ANUNCIO PETRO 02,8664.0,5862.0,41.0,1.0
2,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01,20466.0,17136.0,259.0,20.0
3,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_003_ANUNCIO PETRO 03,7885.0,6254.0,30.0,0.0
4,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_002_ANUNCIO PETRO 02,2032.0,1786.0,14.0,1.0
...,...,...,...,...,...,...,...,...
624,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | SALVOU + ENVIOU MSG | 365D,ADS_001_ANUNCIO PETRO 01,4.0,5.0,0.0,0.0
625,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_003_ANUNCIO PETRO 03,854.0,670.0,9.0,7.0
626,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_001_ANUNCIO PETRO 01,1327.0,925.0,13.0,8.0
627,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 14D,ADS_001_ANUNCIO PETRO 01,169.0,1043.0,9.0,8.0


In [14]:
df_compradores.rename(columns={'email contato': 'email'}, inplace=True)
df_compradores["email"] = df_compradores["email"].str.lower().str.strip()
df_compradores = df_compradores.drop_duplicates(subset='email', keep='first')

df_compradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   593 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


In [15]:
df_pesquisa_compradores_l12 = df_pesquisa_captacao[
    df_pesquisa_captacao["email"].isin(df_compradores["email"])
].copy()

In [16]:
df_pesquisa_compradores_l12.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195 entries, 819 to 27052
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   195 non-null    datetime64[ns]
 1   email                  195 non-null    object        
 2   idade                  195 non-null    object        
 3   nível                  195 non-null    object        
 4   situação profissional  195 non-null    object        
 5   renda                  195 non-null    object        
 6   escolaridade           195 non-null    object        
 7   duvida                 195 non-null    object        
 8   dificuldade            195 non-null    object        
 9   lancamentos            195 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 16.8+ KB


In [17]:
df_pesquisa_compradores_l12.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,lancamentos
819,2025-04-07,23-27,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,O que estudar,Montar um planejamento de estudos,SSP-L12
966,2025-04-07,28-32,Técnica,Funcionário de empresa privada,De 5.000 a 10.000,Nível técnico completo,Assuntos da prova,"Cronograma, oq estudar",SSP-L12
1318,2025-04-07,33-37,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,"Preciso ter algum curso técnico, ou só o ensin...","Não sei por onde começar, nem os conteúdos que...",SSP-L12
270,2025-04-07,28-32,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Em qual cidade irei fazer o concurso?,Não sei o que estudar,SSP-L12
295,2025-04-07,28-32,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino superior completo,Quando vai sair o edital,Foco,SSP-L12


In [18]:
df_compradores_nao_respondentes = df_compradores[
    ~df_compradores["email"].isin(df_pesquisa_captacao["email"])
].copy()

In [19]:
df_compradores_nao_respondentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 398 entries, 0 to 591
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   398 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [20]:
df_pesquisa_compradores_l12['lancamentos'] = 'SSP-L12'

In [21]:
# Colunas de UTM que vamos trazer
colunas_utm = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']

# Fazer o merge trazendo as UTMs para df_pesquisa_captacao
df_pesquisa_captacao = df_pesquisa_captacao.merge(
    df_leads_trafego[['email'] + colunas_utm],
    on='email',
    how='left'  # Mantém todas as respostas da pesquisa, mesmo se não tiver UTM
)

# Fazer o merge trazendo as UTMs para df_pesquisa_compradores_l12
df_pesquisa_compradores_l12 = df_pesquisa_compradores_l12.merge(
    df_leads_trafego[['email'] + colunas_utm],
    on='email',
    how='left'
)

In [22]:
df_pesquisa_captacao.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,lancamentos,utm_source,utm_campaign,utm_medium,utm_content
0,2025-04-07,38-45,Técnica,Já sou funcionário terceirizado do sistema Pet...,De 3.000 a 5.000,Nível técnico completo,Como participar do processo seletivo,Tempo,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_006_ANUNCIO PETRO 06
1,2025-04-07,38-45,Técnica,Desempregado(a),Estou desempregado(a),Nível técnico completo,Assunto específico que cai na prova?,Dificuldade em aprender!,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_006_ANUNCIO PETRO 06
2,2025-04-07,28-32,Técnica,Funcionário público,De 1.000 a 3.000,Ensino médio completo,Nas perguntas,Concentração,SSP-L12,NaN,NaN,NaN,NaN
3,2025-04-07,18-22,Técnica,Desempregado(a),Estou desempregado(a),Ensino médio completo,Como estudar,Constância,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_006_ANUNCIO PETRO 06
4,2025-04-07,23-27,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino médio completo,Qual tempo de curso,Nenhuma,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_006_ANUNCIO PETRO 06


In [23]:
df_pesquisa_compradores_l12.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,lancamentos,utm_source,utm_campaign,utm_medium,utm_content
0,2025-04-07,23-27,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,O que estudar,Montar um planejamento de estudos,SSP-L12,NaN,NaN,NaN,NaN
1,2025-04-07,28-32,Técnica,Funcionário de empresa privada,De 5.000 a 10.000,Nível técnico completo,Assuntos da prova,"Cronograma, oq estudar",SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | SEGUIDORES,ADS_007_ANUNCIO PETRO 07
2,2025-04-07,33-37,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,"Preciso ter algum curso técnico, ou só o ensin...","Não sei por onde começar, nem os conteúdos que...",SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_003_ANUNCIO PETRO 03
3,2025-04-07,28-32,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Em qual cidade irei fazer o concurso?,Não sei o que estudar,SSP-L12,Youtube,,Descricao,
4,2025-04-07,28-32,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino superior completo,Quando vai sair o edital,Foco,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_007_ANUNCIO PETRO 07


In [24]:
# Definir as colunas UTM
colunas_utms = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']

# Contar quantas linhas têm todas as UTMs como NaN
qtd_todas_utms_nan = df_pesquisa_compradores_l12[colunas_utms].isna().all(axis=1).sum()

print(f"Quantidade de linhas com TODAS as UTMs NaN: {qtd_todas_utms_nan}")

Quantidade de linhas com TODAS as UTMs NaN: 57


In [25]:
"""
model = SentenceTransformer("all-MiniLM-L6-v2")

# Mapas distintos por coluna
mapeamentos = {
    'problema_aprender': mapeamento_problema_aprender,
    'profissao': mapeamento_profissoes,
    'fala_outro_idioma': mapeamento_outros_idiomas,
    'motivo_fluencia_espanhol': mapeamento_motivo_fluencia,
    'escolaridade': mapeamento_escolaridade
}

for coluna, mapeamento in mapeamentos.items():
    nome_coluna_categoria = f"{coluna}_categoria"

    # Expandir mapeamento para esta coluna
    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)

    # Categorização com o mapeamento certo
    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(
        df_pesquisa_captacao_lancamentos[coluna],
        mapeamento_expandido,
        mapeamento_embeddings,
        model,
        threshold=0.6,
        desc=coluna
    )
"""

'\nmodel = SentenceTransformer("all-MiniLM-L6-v2")\n\n# Mapas distintos por coluna\nmapeamentos = {\n    \'problema_aprender\': mapeamento_problema_aprender,\n    \'profissao\': mapeamento_profissoes,\n    \'fala_outro_idioma\': mapeamento_outros_idiomas,\n    \'motivo_fluencia_espanhol\': mapeamento_motivo_fluencia,\n    \'escolaridade\': mapeamento_escolaridade\n}\n\nfor coluna, mapeamento in mapeamentos.items():\n    nome_coluna_categoria = f"{coluna}_categoria"\n\n    # Expandir mapeamento para esta coluna\n    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)\n\n    # Categorização com o mapeamento certo\n    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(\n        df_pesquisa_captacao_lancamentos[coluna],\n        mapeamento_expandido,\n        mapeamento_embeddings,\n        model,\n        threshold=0.6,\n        desc=coluna\n    )\n'

In [26]:
"""# Exibir strings completas
pd.set_option("display.max_colwidth", None)

# Colunas categorizadas e respectivas colunas originais
categorias = {
    "problema_aprender": "problema_aprender_categoria",
    "profissao": "profissao_categoria",
    "fala_outro_idioma": "fala_outro_idioma_categoria",
    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",
    "escolaridade": "escolaridade_categoria"
}

# Categorias problemáticas
categorias_problema = ["Outros"]

# Loop para filtrar e exibir como DataFrame formatado
for original_col, categoria_col in categorias.items():
    for problema in categorias_problema:
        print(f"\n=== Valores de '{original_col}' categorizados como '{problema}' ===")

        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema
        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()

        # Envolver cada valor com aspas
        valores.index = [f'"{val}"' for val in valores.index]

        # Converter para DataFrame para visualização completa
        valores_df = valores.reset_index().head(300)
        valores_df.columns = ['valor_original', 'frequencia']

        # Exibir o resultado completo
        print(valores_df.to_string(index=False))
"""

'# Exibir strings completas\npd.set_option("display.max_colwidth", None)\n\n# Colunas categorizadas e respectivas colunas originais\ncategorias = {\n    "problema_aprender": "problema_aprender_categoria",\n    "profissao": "profissao_categoria",\n    "fala_outro_idioma": "fala_outro_idioma_categoria",\n    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",\n    "escolaridade": "escolaridade_categoria"\n}\n\n# Categorias problemáticas\ncategorias_problema = ["Outros"]\n\n# Loop para filtrar e exibir como DataFrame formatado\nfor original_col, categoria_col in categorias.items():\n    for problema in categorias_problema:\n        print(f"\n=== Valores de \'{original_col}\' categorizados como \'{problema}\' ===")\n\n        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema\n        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()\n\n        # Envolver cada valor com aspas\n        valores.index = [f\'"{val}"\' for val in

In [27]:
"""df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([
    'profissao',
    'fala_outro_idioma',
    'motivo_fluencia_espanhol',
    'escolaridade',
    'problema_aprender'
], axis=1, errors='ignore')
"""

"df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([\n    'profissao',\n    'fala_outro_idioma',\n    'motivo_fluencia_espanhol',\n    'escolaridade',\n    'problema_aprender'\n], axis=1, errors='ignore')\n"

In [28]:
from gspread_dataframe import set_with_dataframe
from datetime import datetime

# Carrega as variáveis de ambiente
load_dotenv()

# Pega o caminho de forma segura
credenciais_path = os.getenv("GOOGLE_CREDENTIALS_PATH")

# === Escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# === Autenticar ===
creds = service_account.Credentials.from_service_account_file(
    credenciais_path,
    scopes=scopes
)
client = gspread.authorize(creds)

# === Função para criar nova planilha e carregar dados ===
def criar_planilha_e_enviar(df, nome_base):
    nome_final = nome_base
    
    # Cria nova planilha
    planilha = client.create(nome_final)
    
    # Compartilha com seu e-mail pessoal (aqui você coloca o seu)
    planilha.share('camilobf2@gmail.com', perm_type='user', role='writer')  # <<< Trocar pelo seu email do Gmail

    # Preenche a primeira aba
    aba = planilha.sheet1
    aba.update_title("Dados")
    set_with_dataframe(aba, df)
    
    print(f"✅ Nova planilha criada: {nome_final}")
    print(f"🔗 Link: https://docs.google.com/spreadsheets/d/{planilha.id}/edit")

# === Geração dos arquivos ===
#criar_planilha_e_enviar(df_pesquisa_captacao, "pesquisa_captacao")
#criar_planilha_e_enviar(df_pesquisa_compradores_l12, "pesquisa_aluno")
#criar_planilha_e_enviar(df_leads_invest_trafego, "invest_trafego")

In [29]:
# === Função para atualizar uma planilha existente ===
def atualizar_planilha_existente(df: pd.DataFrame, sheet_id: str, aba_nome: str = "Dados"):
    """
    Atualiza uma aba específica de uma planilha no Google Sheets.
    
    - df: DataFrame com os dados que serão enviados
    - sheet_id: ID da planilha (o que vem na URL depois de "/d/")
    - aba_nome: Nome da aba que será atualizada (default: 'Dados')
    """
    try:
        planilha = client.open_by_key(sheet_id)
        try:
            aba = planilha.worksheet(aba_nome)
        except gspread.WorksheetNotFound:
            aba = planilha.add_worksheet(title=aba_nome, rows="1000", cols="20")
        
        aba.clear()  # Limpa dados antigos
        set_with_dataframe(aba, df)
        
        print(f"✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/{sheet_id}/edit")
    except Exception as e:
        print(f"❌ Erro ao atualizar a planilha: {e}")

# === IDs das planilhas (FIXOS, preencha certinho aqui) ===
id_pesquisa_captacao = "1ukLwu8SoP0U3uirB6w1Ca3TPEITY50c558xNUUN3kj4"
id_pesquisa_aluno = "1GDCAa1fiflDIBnRY9rrdY9ghKwS6DPzxIG9FYGZEsQU"
id_invest_trafego = "1477LAemTkMN1YTFdRJkLMaDLPZEiJ3vvtqwduemHXD4"


# === Atualizar todas as planilhas ===
atualizar_planilha_existente(df_pesquisa_captacao, id_pesquisa_captacao)
atualizar_planilha_existente(df_pesquisa_compradores_l12, id_pesquisa_aluno)
atualizar_planilha_existente(df_leads_invest_trafego, id_invest_trafego)

✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/1ukLwu8SoP0U3uirB6w1Ca3TPEITY50c558xNUUN3kj4/edit


✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/1GDCAa1fiflDIBnRY9rrdY9ghKwS6DPzxIG9FYGZEsQU/edit


✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/1477LAemTkMN1YTFdRJkLMaDLPZEiJ3vvtqwduemHXD4/edit


In [30]:
from datetime import datetime
import pytz

fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

with open("ultima_atualizacao.txt", "w") as f:
    f.write(agora_brasil.strftime("%Y-%m-%d %H:%M:%S"))

In [31]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_captacao.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_captacao[coluna].value_counts(dropna=False))


Coluna: idade
idade
23-27         6212
18-22         5867
28-32         5099
38-45         4748
33-37         4073
46-52         2154
53 ou mais     878
Name: count, dtype: int64

Coluna: nível
nível
Técnica     21982
Superior     7049
Name: count, dtype: int64

Coluna: situação profissional
situação profissional
Funcionário de empresa privada                          12385
Autônomo                                                 6866
Desempregado(a)                                          6632
Funcionário público                                      2071
Empresário                                                594
Já sou funcionário terceirizado do sistema Petrobras      483
Name: count, dtype: int64

Coluna: renda
renda
De 1.000 a 3.000         14580
De 3.000 a 5.000          5384
De 0 a 1.000              3719
Estou desempregado(a)     3359
De 5.000 a 10.000         1713
Mais de 10.000             276
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino médio compl

In [32]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_compradores_l12.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_compradores_l12[coluna].value_counts(dropna=False))


Coluna: idade
idade
23-27         49
28-32         35
38-45         35
33-37         31
46-52         22
18-22         17
53 ou mais     6
Name: count, dtype: int64

Coluna: nível
nível
Técnica     164
Superior     31
Name: count, dtype: int64

Coluna: situação profissional
situação profissional
Funcionário de empresa privada                          99
Desempregado(a)                                         39
Autônomo                                                33
Empresário                                              12
Funcionário público                                     10
Já sou funcionário terceirizado do sistema Petrobras     2
Name: count, dtype: int64

Coluna: renda
renda
De 1.000 a 3.000         79
De 3.000 a 5.000         55
De 5.000 a 10.000        27
Estou desempregado(a)    21
De 0 a 1.000             10
Mais de 10.000            3
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino médio completo         72
Ensino superior completo      61
Nível 

duvida
O que estudar                                                                                 5
O que estudar                                                                                 3
Como passar                                                                                   3
Dificuldade                                                                                   2
Nenhuma                                                                                       2
                                                                                             ..
Não tenho curso técnico, preciso ter pra realizar a prova? E se caso eu passe no corcurso?    1
Assuntos a serem estudados                                                                    1
Se posso ser mesmo chamado mesmo não tendo um diploma de ensino superior                      1
Quando vai ser?                                                                               1
Como estudar com eficiência para 